In [20]:
import pandas as pd
import os
import geopandas as gpd
import re # regular expression
#import pyproj


In [21]:
# GLOBAL VARIABLES

# excel spreadsheet copied to data/udot folder from https://drive.google.com/file/d/1rDXm0ObugGR1zXgWUuVbzWHNt-Xs1xru/view
fnExcelAADTHistory = 'data/udot/AADTHistory.xlsx'

# segment shapefile with AADT and previous forecasts - copied from 'A:/1 - TDM/3 - Model Dev/1 - WF/1 - Official Release/v9x/v9.0/WF TDM v9.0 - official/1_Inputs/6_Segment/Segments_WF - 2023-08-01.shp'
fnSegmentsShapefile = 'data/segments/Segments_WF - 2023-08-01.shp'

# filter by PLANAREA in segments shapefile
filterPlanArea = ['WFRC','MAG'] # must be an array... if only single item, the still include []

# Get AADT and Previous Forecasts from Segment Shapefile

In [22]:
# read in segment shapefile
gdfSegments = gpd.read_file(fnSegmentsShapefile)
display(gdfSegments)

# show columns
#print(gdfSegments.columns.tolist())

,Id,SEGID,BMP,EMP,DISTANCE,FULLNAME,FNCLASS,FC_CODE,FC_GROUP,FC_NAME,...,ForecastSo,Shape_Leng,NAME,Shape_Le_1,OBJECTID,DIRECTION,LINE,ROUTE,Shape_Le_2,geometry
0,0,0006_000.0,0.000,0.665,0.666641,HWY 6,Arterial,3,Arterial,Other Principal Arterial,...,Utah_Statewide_Traffic_Volume_Historic_and_For...,1072.856212,NaN,0.0,0,NaN,NaN,NaN,0.000000,"LINESTRING (236177.700 4327541.250, 237241.180..."
1,0,0006_000.7,0.665,16.022,15.369839,HWY 6,Arterial,3,Arterial,Other Principal Arterial,...,Utah_Statewide_Traffic_Volume_Historic_and_For...,24735.407823,NaN,0.0,0,NaN,NaN,NaN,0.000000,"LINESTRING (237241.180 4327399.720, 238150.600..."
2,0,0006_016.0,16.022,46.017,30.001961,HWY 6,Arterial,3,Arterial,Other Principal Arterial,...,Utah_Statewide_Traffic_Volume_Historic_and_For...,48283.572122,NaN,0.0,0,NaN,NaN,NaN,0.000000,"LINESTRING (261403.900 4327045.400, 261409.000..."
3,0,0006_046.0,46.017,60.218,14.194306,HWY 6,Arterial,3,Arterial,Other Principal Arterial,...,Utah_Statewide_Traffic_Volume_Historic_and_For...,22843.567118,NaN,0.0,0,NaN,NaN,NaN,0.000000,"LINESTRING (305380.160 4325741.850, 305784.500..."
4,0,0006_060.2,60.218,77.545,17.323237,HWY 6,Arterial,3,Arterial,Other Principal Arterial,...,Utah_Statewide_Traffic_Volume_Historic_and_For...,27879.103631,NaN,0.0,0,NaN,NaN,NaN,0.000000,"LINESTRING (324618.000 4337936.900, 324647.400..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8722,0,UTA_7132,0.000,0.000,0.178270,NaN,NaN,0,NaN,NaN,...,NaN,0.042997,NaN,0.0,7,north,North/South TRAX,BGR,0.002584,"LINESTRING (424262.689 4508606.237, 424262.070..."
8723,0,UTA_7316,0.000,0.000,1.150144,NaN,NaN,0,NaN,NaN,...,NaN,0.183122,NaN,0.0,6,east,Mid Jordan TRAX,R,0.021304,"LINESTRING (420773.384 4495314.959, 420666.117..."
8724,0,UTA_7320,0.000,0.000,0.609642,NaN,NaN,0,NaN,NaN,...,NaN,0.183122,NaN,0.0,6,east,Mid Jordan TRAX,R,0.011282,"LINESTRING (419033.244 4494684.053, 418947.477..."
8725,0,UTA_7328,0.000,0.000,1.352924,NaN,NaN,0,NaN,NaN,...,NaN,0.183122,NaN,0.0,6,east,Mid Jordan TRAX,R,0.023805,"LINESTRING (415002.731 4492771.270, 414816.767..."


In [23]:
# filter segments by PLANAREA
_df = gdfSegments[gdfSegments['PLANAREA'].isin(filterPlanArea)].copy()

# get columns with AADT at beginning of name
aadt_columns = [col for col in _df.columns if col.startswith('AADT')]

# get dataframe for just segids and aadt columns
_df = _df[['SEGID'] + aadt_columns]

# melt to make long
_df = _df.melt(id_vars='SEGID', var_name='YEAR', value_name='AADT')

# filter out zero volumes
_df = _df[_df['AADT']>0]

# get YEAR from string
_df['YEAR'] = _df['YEAR'].str[-4:].astype(int)

# set AADT to int
_df['AADT'] = _df['AADT'].astype(int)

# set source
_df['SOURCE'] = fnSegmentsShapefile.split('/')[-1]

dfAadtFromSegments = _df

dfAadtFromSegments

,SEGID,YEAR,AADT,SOURCE
0,0006_146.9,2019,1517,Segments_WF - 2023-08-01.shp
1,0006_149.9,2019,2441,Segments_WF - 2023-08-01.shp
2,0006_150.6,2019,2441,Segments_WF - 2023-08-01.shp
3,0006_152.6,2019,2417,Segments_WF - 2023-08-01.shp
4,0006_152.9,2019,3759,Segments_WF - 2023-08-01.shp
...,...,...,...,...
164120,2899_002.4,1981,2635,Segments_WF - 2023-08-01.shp
164121,2899_003.5,1981,2635,Segments_WF - 2023-08-01.shp
164122,2899_004.1,1981,850,Segments_WF - 2023-08-01.shp
165368,0186_004.6,1981,34200,Segments_WF - 2023-08-01.shp


In [24]:
# filter segments by PLANAREA
_df = gdfSegments[gdfSegments['PLANAREA'].isin(filterPlanArea)].copy()

# get columns with F at beginning of name and last four characters are numeric
forecast_columns = [col for col in _df.columns if col.startswith('F') and re.match('.*\d{4}$', col)]

# get dataframe for just segids and aadt columns
_df = _df[['SEGID'] + forecast_columns]

# melt to make long
_df = _df.melt(id_vars='SEGID', var_name='YEAR', value_name='FORECAST')

# filter out zero volumes
_df = _df[_df['FORECAST']>0]

# get YEAR from string
_df['YEAR'] = _df['YEAR'].str[-4:].astype(int)

# set AADT to int
_df['FORECAST'] = _df['FORECAST'].astype(int)

# set source
_df['SOURCE'] = fnSegmentsShapefile.split('/')[-1]

dfForecastsFromSegments = _df

dfForecastsFromSegments

,SEGID,YEAR,FORECAST,SOURCE
0,0006_146.9,2050,2000,Segments_WF - 2023-08-01.shp
1,0006_149.9,2050,4300,Segments_WF - 2023-08-01.shp
2,0006_150.6,2050,4300,Segments_WF - 2023-08-01.shp
3,0006_152.6,2050,4600,Segments_WF - 2023-08-01.shp
4,0006_152.9,2050,6600,Segments_WF - 2023-08-01.shp
...,...,...,...,...
8611,2107_003.5,2040,7700,Segments_WF - 2023-08-01.shp
8612,2094_002.5,2040,11000,Segments_WF - 2023-08-01.shp
8613,2040_007.2,2040,9300,Segments_WF - 2023-08-01.shp
8614,2168_003.1,2040,9000,Segments_WF - 2023-08-01.shp


# Get Historic Data from AADTHistory UDOT Spreadsheet

In [25]:
#import excel spreadsheet
xl_file1 = pd.ExcelFile(fnExcelAADTHistory)
dfs1 = {sheet_name: xl_file1.parse(sheet_name) for sheet_name in xl_file1.sheet_names}

c:\Users\bhereth\Anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [26]:
#show sheet
display(xl_file1.sheet_names)
dfs1['AADT_2021']

['AADT_2021', 'AADT_Rounded', 'AADT_Unrounded']

,ROUTE_ID,FROM_MEASURE,TO_MEASURE,AADT_2021
0,0006PM,0.000,46.038,474
1,0006PM,46.038,77.556,424
2,0006PM,77.556,82.897,609
3,0006PM,82.897,83.911,2272
4,0006PM,83.911,87.694,3852
...,...,...,...,...
4568,0015PC29402,0.000,0.199,8886
4569,0015PC29501,0.000,0.166,1000
4570,0015PC30554,0.000,1.135,35025
4571,0092NC00101,0.000,3.294,12201


In [27]:
# PREPARE 2021 DATA

df2021 = dfs1['AADT_2021']

df2021['YEAR'] = 2021
df2021.rename(columns={'AADT_2021':'AADT'}, inplace=True)
df2021

,ROUTE_ID,FROM_MEASURE,TO_MEASURE,AADT,YEAR
0,0006PM,0.000,46.038,474,2021
1,0006PM,46.038,77.556,424,2021
2,0006PM,77.556,82.897,609,2021
3,0006PM,82.897,83.911,2272,2021
4,0006PM,83.911,87.694,3852,2021
...,...,...,...,...,...
4568,0015PC29402,0.000,0.199,8886,2021
4569,0015PC29501,0.000,0.166,1000,2021
4570,0015PC30554,0.000,1.135,35025,2021
4571,0092NC00101,0.000,3.294,12201,2021


In [28]:
# GET AADTS FOR YEARS PRIOR TO 2021

# melt AADT columns to rows
df = dfs1['AADT_Unrounded']

# get all columns that start with 'AADT'
aadt_cols = [col for col in df.columns if col.startswith('AADT')]

# define the id columns that you want to keep
id_vars = ['Route', 'Beg MP', 'End MP']

# melt the DataFrame
dfUnder2021 = df.melt(id_vars=id_vars, value_vars=aadt_cols, var_name='YEAR', value_name='AADT')

# replace 'AADT' in the 'YEAR' column and convert to integer
dfUnder2021['YEAR'] = dfUnder2021['YEAR'].str.replace('AADT', '').astype(int)

# rename columns to match pre-2021 data
dfUnder2021.rename(columns={'Route':'ROUTE_ID','Beg MP':'FROM_MEASURE','End MP':'TO_MEASURE'}, inplace=True)

display(dfUnder2021)

,ROUTE_ID,FROM_MEASURE,TO_MEASURE,YEAR,AADT
0,0006PM,0.000,46.038,2020,430
1,0006PM,46.038,77.556,2020,385
2,0006PM,77.556,82.897,2020,552
3,0006PM,82.897,83.911,2020,2061
4,0006PM,83.911,87.694,2020,3409
...,...,...,...,...,...
181475,3468PM,0.000,5.125,1981,0
181476,3469PM,0.000,6.930,1981,0
181477,3470PM,0.000,1.039,1981,0
181478,3478PM,0.000,2.040,1981,0


In [29]:
# combine 2021 and under 2021
dfAADT = pd.concat([dfUnder2021,df2021])

# melt for easier joining
dfAADT = pd.pivot_table(dfAADT,index=('ROUTE_ID','FROM_MEASURE','TO_MEASURE'),columns='YEAR',values='AADT')

dfAADT.reset_index(inplace=True)
dfAADT.fillna('0',inplace=True)

# Filter rows with 'ROUTE_ID' containing 'PM' - Positive Direction & Mainline?
dfAADT_filtered = dfAADT[dfAADT['ROUTE_ID'].str.contains('PM')].copy()

# Trim 'ROUTE_ID' down to the first four characters in order to match with SEGID
dfAADT_filtered['ROUTE_ID'] = dfAADT_filtered['ROUTE_ID'].str[:4]

dfAADT_filtered

YEAR,ROUTE_ID,FROM_MEASURE,TO_MEASURE,1981,1982,1983,1984,1985,1986,1987,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,0006,0.000,46.038,325.0,335.0,430.0,580.0,585.0,585.0,595.0,...,325.0,330.0,350.0,375.0,399.0,409.0,412.0,415.0,430.0,474.0
1,0006,46.038,77.556,325.0,335.0,430.0,580.0,585.0,585.0,595.0,...,340.0,340.0,360.0,390.0,412.0,366.0,369.0,372.0,385.0,424.0
2,0006,77.556,82.897,520.0,535.0,630.0,780.0,790.0,790.0,800.0,...,420.0,420.0,450.0,480.0,512.0,525.0,529.0,533.0,552.0,609.0
3,0006,82.897,83.911,0.0,1350.0,1450.0,1650.0,1675.0,1600.0,1525.0,...,1570.0,1575.0,1675.0,1800.0,1913.0,1961.0,1975.0,1991.0,2061.0,2272.0
4,0006,83.911,87.694,0.0,2650.0,2750.0,3000.0,3050.0,2920.0,2790.0,...,3270.0,3340.0,3495.0,3660.0,3846.0,3468.0,3496.0,3573.0,3409.0,3852.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4755,3468,0.000,5.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2015.0,2085.0,2120.0,2245.0,2354.0,2452.0,2550.0,2581.0,2578.0,2764.0
4756,3469,0.000,6.930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,490.0,505.0,515.0,545.0,572.0,333.0,346.0,350.0,350.0,375.0
4757,3470,0.000,1.039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1653.0,1694.0,1724.0,1540.0,1669.0
4758,3478,0.000,2.040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1055.0,1090.0,1110.0,1175.0,1232.0,865.0,900.0,911.0,910.0,976.0


In [30]:
# CHECK FOR DUPLICATES

# Define the columns you want to check for duplicates
columns_to_check = ['ROUTE_ID', 'FROM_MEASURE', 'TO_MEASURE']

# Use the duplicated method to find duplicates in those columns
duplicates = dfAADT_filtered.duplicated(subset=columns_to_check, keep=False)

# Filter the DataFrame to only include the duplicates
dfAADT_duplicates = dfAADT_filtered[duplicates]

display(dfAADT_duplicates)

YEAR,ROUTE_ID,FROM_MEASURE,TO_MEASURE,1981,1982,1983,1984,1985,1986,1987,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021


In [31]:
# JOIN DATA BACK TO SEGMENTS FILE AND FILTER WITH MIDPOINT_MP BETWEEN BMP AND EMP

# filter by plan area
_df = gdfSegments[gdfSegments['PLANAREA'].isin(filterPlanArea)].copy()

# calculate midpoint for 
_df['MIDPOINT_MP'] = (_df['BMP'] + _df['EMP']) / 2
_df['ROUTE_ID'] = _df['SEGID'].str.split('_').str[0]

# get only data needed to join
_df = _df[['SEGID','ROUTE_ID','MIDPOINT_MP']]

# create temp dataframe to merge to segments data
_df2 = pd.DataFrame.merge(_df, dfAADT_filtered, on='ROUTE_ID')

# filter for only segments with midpoints between AADT FROM_MEASURE and TO_MEASURE
_df2 = _df2[_df2['MIDPOINT_MP'].between(_df2['FROM_MEASURE'],_df2['TO_MEASURE'])]

# give me only columns with years
cols_to_keep = [col for col in _df2.columns if str(col).isdigit() and 1981 <= int(col) <= 2021]

# filter by only segid and columns with years
_df2 = _df2[['SEGID']+cols_to_keep]

# melt to make table long
_df2 = _df2.melt(id_vars='SEGID',var_name="YEAR",value_name='AADT')

# convert AADT to int
_df2['AADT'] = _df2['AADT'].astype(int)

# add source
_df2['SOURCE'] = fnExcelAADTHistory.split('/')[-1]

# filter out an zero data
_df2 = _df2[_df2['AADT']>0]

dfAadtFromHistoric = _df2

dfAadtFromHistoric

,SEGID,YEAR,AADT,SOURCE
0,0006_146.9,1981,975,AADTHistory.xlsx
1,0006_149.9,1981,1200,AADTHistory.xlsx
2,0006_150.6,1981,1200,AADTHistory.xlsx
3,0006_152.6,1981,1800,AADTHistory.xlsx
4,0006_152.9,1981,1800,AADTHistory.xlsx
...,...,...,...,...
160387,3465_000.0,2021,493,AADTHistory.xlsx
160388,3466_000.0,2021,7681,AADTHistory.xlsx
160389,3466_000.1,2021,7681,AADTHistory.xlsx
160390,3466_000.3,2021,7681,AADTHistory.xlsx


# Wrap up and export

In [32]:
dfAadt = pd.concat([dfAadtFromSegments,dfAadtFromHistoric])
dfAadt = dfAadt.sort_values(by=['SEGID','SOURCE','YEAR'])
dfAadt

,SEGID,YEAR,AADT,SOURCE
0,0006_146.9,1981,975,AADTHistory.xlsx
3912,0006_146.9,1982,1000,AADTHistory.xlsx
7824,0006_146.9,1983,965,AADTHistory.xlsx
11736,0006_146.9,1984,1240,AADTHistory.xlsx
15648,0006_146.9,1985,1340,AADTHistory.xlsx
...,...,...,...,...
21615,WFRC_8263,2015,13770,Segments_WF - 2023-08-01.shp
17259,WFRC_8263,2016,14498,Segments_WF - 2023-08-01.shp
12903,WFRC_8263,2017,6489,Segments_WF - 2023-08-01.shp
8547,WFRC_8263,2018,6554,Segments_WF - 2023-08-01.shp


In [33]:
# CHECK NON-NUMERIC SEGIDs!!
dfAadt['SEGID'][dfAadt['SEGID'].str.match(r'[^\d]')].drop_duplicates()


12455     MAG_6017
3887      MAG_6164
12899     MAG_6254
43394    WFRC_8004
73878    WFRC_8220
25973    WFRC_8221
25974    WFRC_8222
25975    WFRC_8223
43393    WFRC_8262
43395    WFRC_8263
Name: SEGID, dtype: object

In [34]:
dfAadt.rename(columns={'AADT':'obsAADT'})

# export AADT
dfAadt.to_csv('intermediate/aadt.csv',index=False)

# export AADT sources
dfAadt[['SOURCE']].drop_duplicates().to_csv('intermediate/aadt-sources.csv',index=False)

# export Forecasts
dfForecastsFromSegments.to_csv('intermediate/previous-forecasts.csv',index=False)

# export AADT sources
dfForecastsFromSegments[['SOURCE']].drop_duplicates().to_csv('intermediate/previous-forecasts-sources.csv',index=False)